In [4]:
!pip install datasets evaluate transformers[sentencepiece] peft tqdm 
!pip install scikit-learn

!pip install --upgrade peft


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.3.2-cp311-cp311-win_amd64.whl (9.2 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)



[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"
base_model = "meta-llama/Llama-2-7b-hf"
peft_model = "FinGPT/fingpt-mt_llama2-7b_lora"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map=device)

model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()

c:\Users\Carlos\Documents\git\llms\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.67s/it]


In [2]:
instruction = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."
news = "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m" 

instruction = f"{instruction} \n{news} \nAnswer: "
print(instruction)

model_input = tokenizer(instruction, return_tensors="pt")
model_input = model_input.to(device)

res = model.generate(**model_input)

What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. 
For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m 
Answer: 


In [ ]:
res_sentences = [tokenizer.decode(i) for i in res]
res_sentences

["<s> What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. \nFor the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m \nAnswer:  neutral</s>"]

In [ ]:
decoded = tokenizer.batch_decode(res)
decoded

["<s> What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. \nFor the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m \nAnswer:  neutral</s>"]

In [18]:
from sklearn.metrics import accuracy_score,f1_score
from datasets import load_dataset
from tqdm import tqdm
import torch


def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

def change_target(x):
    if 'positive' in x or 'Positive' in x:
        return 0
    elif 'negative' in x or 'Negative' in x:
        return 2
    else:
        return 1


def test_fiqa(model, tokenizer, batch_size = 8):
    dataset = load_dataset('pauri32/fiqa-2018')
    dataset = dataset["test"]
    dataset = dataset.to_pandas()

    # only to validate function
    #dataset = dataset.head(2)

    dataset["output"] = dataset['label']
    dataset["instruction"] = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."

    dataset = dataset[['sentence', 'output', 'instruction']]
    dataset.columns = ['input', 'output', 'instruction']
    dataset[['context', 'target']] = dataset.apply(format_example, axis = 1, result_type="expand")

    # print example
    print(f"\n\nPrompt example:\n{dataset['context'][1]}\n\n")

    context = dataset['context'].tolist()
    total_steps = dataset.shape[0]//batch_size + 1
    print(f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text_list = []

    for i in tqdm(range(total_steps)):
        tmp_context = context[i* batch_size:(i+1)* batch_size]
        tokenizer.pad_token = tokenizer.eos_token
        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True)
        for k in tokens.keys():
            tokens[k] = tokens[k].cuda()
        
        res = model.generate(**tokens)
        res_sentences = tokenizer.batch_decode(res)
        out_text = [o.split("Answer: ")[1] for o in res_sentences]
        out_text_list += out_text
        torch.cuda.empty_cache()

    dataset["out_text"] = out_text_list
    dataset["new_out"] = dataset["out_text"].apply(change_target)

    acc = accuracy_score(dataset["target"], dataset["new_out"])
    f1_macro = f1_score(dataset["target"], dataset["new_out"], average = "macro")
    f1_micro = f1_score(dataset["target"], dataset["new_out"], average = "micro")
    f1_weighted = f1_score(dataset["target"], dataset["new_out"], average = "weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    return dataset

In [19]:
test_fiqa(model, tokenizer)
#test_fiqa(None, None)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?
Answer: 


Total len: 150. Batchsize: 8. Total steps: 19


  5%|▌         | 1/19 [00:27<08:16, 27.59s/it]

['</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' negative</s>', ' positive</s>']


 11%|█         | 2/19 [00:56<08:06, 28.60s/it]

['</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' negative</s>', ' positive</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' positive</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s> No</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

...

 95%|█████████▍| 18/19 [08:51<00:28, 28.15s/it]

['</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' negative</s>', ' positive</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' positive</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s> No</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

100%|██████████| 19/19 [09:17<00:00, 29.36s/it]

['</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' negative</s>', ' positive</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', ' positive</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s> No</s>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><unk>', '</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></

,input,output,instruction,context,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna giv...,0,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,0,</s></s></s></s></s></s></s></s></s></s></s></...,1
1,@gakrum nice chart shows distinctive down chan...,2,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,2,</s></s></s><unk>,1
2,Japan's Asahi to submit bid next week for SABM...,0,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,0,</s></s></s></s></s></s></s></s></s></s></s></...,1
3,"Tesla Motors recalls 2,700 Model X SUVs $TSLA ...",2,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,2,</s></s></s></s></s><unk>,1
4,CRH's concrete bid for Holcim Lafarge assets,0,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,0,</s></s></s></s></s></s></s></s></s></s></s></...,1
...,...,...,...,...,...,...,...
145,Intertek swings to ÃÂ£347 mln loss on oil's s...,2,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,2,</s></s></s></s></s></s></s></s></s></s></s></...,1
146,RT @jan $ARNA Don't think buyout rumor strong ...,1,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,1,neutral</s>,1
147,Barclays appoints JPMorgan's Paul Compton as n...,1,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,1,</s></s></s></s></s></s></s></s></s></s></s></...,1
148,$AAPL Now I'm glad I got stopped out of my $11...,1,What is the sentiment of this news? Please cho...,Instruction: What is the sentiment of this new...,1,</s></s><unk>,1
